In [2]:
# ============ IMPORTAÇÕES ============ #
from time import sleep
import requests

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# ============ CONSTANTES ============ #
# BASE_DIR = Path(__file__).parent
BASE_DIR = os.getcwd()
CHAVE_API = os.getenv('CHAVE_API')

In [18]:
# ============ Configuração Básica ============ #
chrome_options = Options()
chrome_options.add_argument(r"user-data-dir=" + str(BASE_DIR) + '/profile/whatsapp')

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://web.whatsapp.com/')
sleep(10)

wait = WebDriverWait(driver=driver, timeout=60)

In [4]:
# ============ Configurações da API ============ #
# ===== EDITARCODIGO ===== #
agent = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
api = requests.get(url=f"https://editacodigo.com.br/index/api-whatsapp/{CHAVE_API}",  headers=agent)
sleep(1)
api = api.text
api = api.split(".n.")

In [5]:
# ============ Objetos ============ #
bolinha_verde = api[3].strip()  # _ahlk
contato_cliente = api[4].strip()  # //*[@id="main"]/header/div[2]/div/div/div/span
msg_cliente = api[6].strip() # '_amjv'
caixa_msg1 = api[7].strip() # 'div[title="Digite uma mensagem"]'

caixa_msg2 = api[5].strip()  # //*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p
caixa_pesquisa = api[8].strip() # "div[title='Caixa de texto de pesquisa']"


# ============ Funções ============ #

In [6]:
def click_notificacao():
    bolinha = driver.find_element(By.CLASS_NAME, bolinha_verde)
    bolinha = driver.find_elements(By.CLASS_NAME, bolinha_verde)
    click_bolinha = bolinha[-1]
    action = webdriver.common.action_chains.ActionChains(driver)
    action.move_to_element_with_offset(click_bolinha, 0, -20)
    action.click().perform()
    action.click().perform()

In [8]:
def pegar_telefone():
    pega_tel_cliente = driver.find_element(By.XPATH, contato_cliente)
    telefone_cliente = pega_tel_cliente.text

    return telefone_cliente

In [7]:
def pegar_n_mensagem(n):
    todas_as_msg = driver.find_elements(By.CLASS_NAME, msg_cliente)
    lista_msg_texto = [e.text for e in todas_as_msg]
    msg = lista_msg_texto[-n]
    msg = str(msg).split('\n')[0]

    return msg
#pega a mensagem antes de enviar

In [9]:
def mensagem_qualquer(msg1):
    campo_de_texto = driver.find_element(By.XPATH, caixa_msg1)
    campo_de_texto.click()
    # Banco de respostas
    
    sleep(3)
    campo_de_texto.send_keys(msg1, Keys.ENTER)

In [16]:
def mensaguem_php():
    msg = pegar_n_mensagem(1)
    tel = pegar_telefone()
    resposta = requests.get('http://localhost/BotBarbeariaZap/index.php?', 
                            params={'msg': msg, 'telefone':tel}, headers=agent)
    sleep(2)
    resposta = resposta.text
    print(resposta)

In [ ]:
def bem_vindo():
    campo_de_texto = driver.find_element(By.XPATH, caixa_msg1)
    campo_de_texto.click()
    sleep(1)
    campo_de_texto.send_keys("Bem vindo a Barbearia dois irmão", Keys.ENTER)

    campo_de_texto = driver.find_element(By.XPATH, caixa_msg1)
    campo_de_texto.click()
    sleep(1)
    campo_de_texto.send_keys("R$20,00....Corte Básico", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$25,00....Corte Navalhado", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$5,00......Só o pezinho", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$5,00......Sobrancelha", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$10,00....Barba", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$30,00....Combo cabelo, barba e Sobrancelha", Keys.SHIFT + Keys.ENTER)
    campo_de_texto.send_keys("R$5,00......Desenhos e Listras", Keys.ENTER)

    campo_de_texto = driver.find_element(By.XPATH, caixa_msg1)
    campo_de_texto.click()
    sleep(1)
    campo_de_texto.send_keys("Rua Quize de Novembro, 378 - Praia de Mauá" + Keys.ENTER)

# ============ Bot ============ #

In [ ]:
def bot():
    try:
        click_notificacao()
        contato = pegar_telefone()
        mensagem_qualquer(f'Olá {contato}')

        # FECHAR CONTATO
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    except:
        print('Buscando novas mensagens não lidas!')
        sleep(5)

# ============ Start ============ #

In [ ]:
while True:
    bot()